# Suivi d’Objets par Frame

Ce notebook implémente une méthode de suivi d’objets basée sur OpenCV et le tracker CSRT. La séquence d’images est lue à partir d’un dossier (`frames`) et les détections initiales sont définies dans le fichier `init.txt` (chaque ligne doit contenir `id,x,y,w,h`).

L’approche utilisée est le *tracking-by-detection* où l’on initialise un tracker pour chaque objet détecté dans la première image et l’on met à jour leur position à chaque frame.

In [2]:
import cv2
import os
import glob

def lire_init(init_path):
    """
    Lit le fichier init.txt.
    Chaque ligne doit contenir : id,x,y,w,h
    Retourne une liste de tuples (id, (x, y, w, h))
    """
    init_data = []
    with open(init_path, 'r') as f:
        for ligne in f:
            ligne = ligne.strip()
            if ligne == "" or ligne.startswith("#"):
                continue
            try:
                parts = ligne.split(',')
                obj_id = parts[0].strip()
                x = int(float(parts[1].strip()))
                y = int(float(parts[2].strip()))
                w = int(float(parts[3].strip()))
                h = int(float(parts[4].strip()))
                init_data.append((obj_id, (x, y, w, h)))
            except Exception as e:
                print(f"Erreur lors de la lecture de la ligne: {ligne} ({e})")
    return init_data

def main():
    # Chemins vers les données (à adapter selon votre environnement)
    frames_folder = "TP3_data/frames"  # dossier contenant les images (formats jpg, png, etc.)
    init_path = "./TP3_data/init.txt"    # fichier contenant les boîtes initiales

    # Vérification de l'existence du dossier et du fichier d'initialisation
    if not os.path.isdir(frames_folder):
        print(f"Erreur : le dossier {frames_folder} n'existe pas.")
        return
    if not os.path.isfile(init_path):
        print(f"Erreur : le fichier {init_path} n'existe pas.")
        return

    # Récupérer et trier la liste des images
    image_files = sorted(glob.glob(os.path.join(frames_folder, "*.jpg")))
    if len(image_files) == 0:
        print("Aucune image trouvée dans le dossier spécifié.")
        return

    # Lecture de la première image pour initialiser les trackers
    frame = cv2.imread(image_files[0])
    if frame is None:
        print(f"Impossible de lire la première image : {image_files[0]}")
        return

    # Lecture des détections initiales
    init_boxes = lire_init(init_path)
    if len(init_boxes) == 0:
        print("Aucune détection initiale trouvée dans le fichier init.txt")
        return

    # Initialisation du multi-tracker (ici avec le tracker CSRT)
    multi_tracker = cv2.MultiTracker_create()
    # Stockage des IDs pour assurer la cohérence lors du dessin
    ids = []
    for obj_id, bbox in init_boxes:
        tracker = cv2.TrackerCSRT_create()
        multi_tracker.add(tracker, frame, bbox)
        ids.append(obj_id)

    # Boucle sur toutes les images de la séquence
    for image_path in image_files:
        frame = cv2.imread(image_path)
        if frame is None:
            print(f"Erreur lors de la lecture de l'image : {image_path}")
            continue

        # Mise à jour de tous les trackers pour la frame courante
        success, boxes = multi_tracker.update(frame)
        if not success:
            print(f"Erreur de suivi pour l'image : {image_path}")
            continue

        # Affichage des boîtes avec leur identifiant
        for i, new_box in enumerate(boxes):
            x, y, w, h = [int(v) for v in new_box]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {ids[i]}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

        # Affichage de l'image suivie
        cv2.imshow("Suivi des tasses", frame)
        key = cv2.waitKey(30) & 0xFF
        if key == 27:  # Appuyer sur 'Echap' pour quitter
            break

    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

Erreur lors de la lecture de la ligne: 0 1 877 427 419 510 (list index out of range)
Aucune détection initiale trouvée dans le fichier init.txt


### Instructions

1. **Préparer vos données** : Placez vos images dans un dossier nommé `frames` (vous pouvez adapter le chemin dans le code) et assurez-vous que le fichier `init.txt` est dans le même répertoire que ce notebook.
2. **Exécuter le Notebook** : Lancez chaque cellule pour initialiser le suivi et visualiser les images avec les objets suivis.
3. **Personnalisation** : Vous pouvez modifier le code pour intégrer d’autres trackers ou améliorer l’association et le suivi selon vos besoins.

Appuyez sur `Echap` dans la fenêtre d’affichage pour interrompre le suivi.